In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density'

In [ ]:
url_pop_html = requests.get(url)

In [ ]:
url_pop_bs = bs(url_pop_html.text)

In [ ]:
tables = url_pop_bs.find_all('table',class_='wikitable')

In [ ]:
pop_table = tables[0]

In [ ]:
fields = [th.text.strip() for th in pop_table.find_all('tr')[0].find_all('th')]

In [ ]:
fields

In [ ]:
trs = pop_table.find_all('tr')[2:-2]

In [ ]:
rows = []

In [ ]:
for tr in trs:
    tds = tr.find_all('td')
    if not tds:
        continue
    data = [td.text.strip() for td in tds]
    del data[2]
    del data[4]
    rows.append({key:value for key,value in zip(fields,data)})

In [ ]:
pop_df = pd.DataFrame(rows)

In [ ]:
pop_df['Country (or dependent territory)'] = pop_df['Country (or dependent territory)'].apply(lambda x: x.split(' (', 1)[0].split('[',1)[0])

In [ ]:
pop_df

In [ ]:
eq_table = pd.read_csv('../Data/04_deadly_eq_mag_cleaned.csv', index_col=False)

In [ ]:
eq_table['country'] = eq_table['present-day country and link to wikipedia article'].apply(lambda x: x.split(' (', 1)[0])

In [ ]:
eq_table.at[1033, 'country'] = 'Venezuela'

In [ ]:
eq_table['country_join'] = eq_table['country'].str.lower()

In [ ]:
pop_df['country_join'] = pop_df['Country (or dependent territory)'].str.lower()

In [ ]:
eq_table_pop = eq_table.merge(pop_df, on=['country_join'] )

In [ ]:
eq_table_pop

In [ ]:
eq_table_pop['Density'] = pd.to_numeric(eq_table_pop['Density'].replace(r'[^\d.]', '', regex=True))

In [ ]:
eq_table_pop.corr()

In [ ]:
corr = eq_table_pop.corr()

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=bool))

In [ ]:
f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});